In [1]:
##三大法人
import requests
import pandas as pd
import re
def str_to_int(text):
    n = 0
    text = text.strip()
    t2 = re.sub(",","",text)
    if "-" in t2:
        n = -1*int(t2.strip('-'))
    else:
        n = int(t2)
    return n
def get_sanda_db(date='20200916'):
    #https://www.twse.com.tw/fund/T86?response=csv&date=20200916&selectType=01
    link = "https://www.twse.com.tw/fund/T86?response=csv&date="+date+"&selectType=01"
    #https://www.twse.com.tw/fund/TWT47U?response=csv&date=20200901&selectType=01
    r = requests.get(link)
    if r.status_code == 200:
        text = r.text
        lines = re.split("\r\n",text)
        lines = lines[1:-11]
        header = lines[0]
        data = lines[1:]
        lol = []
        for e in data:
            t = e.strip( '" ",="')
            t = t.strip( '"*",="')
            items = t.split('","')
            items[0] = items[0].strip()
            for c in range(2,len(items)) :
                items[c] = str_to_int(items[c])
            lol.append(items)
        headers = header.split('","')
        headers[0] = headers[0].strip('"')
        headers[-1] = headers[-1].strip('",')
        #print(headers,len(headers))
        #print(lol[-1],len(lol[-1]))
        df  =pd.DataFrame(lol,columns=headers)
        #df.index = df['證券代號']
    else:
        print("error",r.status_code)
        df = None
    return df

In [6]:
link = "https://www.twse.com.tw/fund/T86?response=csv&date=20201023&selectType=ALL"
r = requests.get(link)

In [81]:
import datetime

In [128]:
def get_sanda_df(date):

    r = requests.get(link)
    if r.status_code == 200:
        import functools
        db = []
        lines = r.text.split("\r\n")
        #print("line",lines)
        if len(lines) <=2 :return None
        header = lines[1].split('","')[:-1]
        header  = list(map(lambda x: x.replace(',',"").replace('"',"").replace(" ","") , header))
        #print(header)
        for e in lines[2:-10]:
            #print(e)
            #print(e.split(",")[:-1])
            #db.append(e.split('","')[:-1])
            rows = e.split('","')[:-1]
            row_data =list(map(lambda x: x.replace(',',"").replace('"',"").replace(" ","") , rows))
            db.append(row_data)
        #print(db)
        df = pd.DataFrame(db)
        #df['date']
        df.columns = header
        df['datetime']= date
        #print("show",date)
        return df
    else:
        print("fail :", r.status_code,link)

In [129]:
get_sanda_df(datetime.datetime(2020,1,1))

In [153]:
#get_sanda_df(datetime.datetime(2020,10,23))

In [154]:
date = datetime.datetime(2019,1,1)
end =  datetime.datetime(2019,2,1)
import numpy
def build_sd_date(start , end ):
    date = start
    df_sd = pd.DataFrame()
    while date!= end :
        df = get_sanda_df(date.date())
        #print(date)
        if type(df) == type(df_sd):
            df_sd = pd.concat([df_sd, df])
        date += datetime.timedelta(days = 1)
    
    for e in df_sd.columns:
        #print(e)
        if e != None and e != '證券名稱' and e != '證券代號'and e != '自營商買進股數(避險)'and e != 'datetime':
            df_sd[e] = df_sd[e].astype(np.int32)
    return df_sd

d = build_sd_date(date,end)

show 2019-01-02
show 2019-01-03
show 2019-01-04
show 2019-01-07
show 2019-01-08
show 2019-01-09
show 2019-01-10
show 2019-01-11
show 2019-01-14
show 2019-01-15
show 2019-01-16
show 2019-01-17
show 2019-01-18
show 2019-01-21
show 2019-01-22
show 2019-01-23
show 2019-01-24
show 2019-01-25
show 2019-01-28
show 2019-01-29
show 2019-01-30


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [152]:
d['自營商賣出股數(避險)'].dtype

dtype('int32')

In [53]:
    date='20200916'
    link = "https://www.twse.com.tw/fund/TWT47U?response=csv&date="+date+"&selectType=01"
    #https://www.twse.com.tw/fund/TWT47U?response=csv&date=20200901&selectType=01
    r = requests.get(link)

In [ ]:
todo:
    1. build an increasing database
    2. interface for date/stock_id